# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closure\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
from collections import OrderedDict, Counter
import functools

In [2]:
c = OrderedDict()

In [20]:
class LruCache(object):
    def __init__(self, max_count):
        self.max_count = max_count
        self.items = OrderedDict()

    def __getitem__(self, key):
        try:
            value = self.items[key]
            return value
        except KeyError:
            raise

    def __setitem__(self, key, value):
        if (len(self.items) > self.max_count):
            self.items.popitem()
        self.items[key] = value
        
    def __call__(self, *args, **kwargs):
        pass

    pass

#### Реализуйте декоратор

In [21]:
def cached(max_count):
    def decorator(func):
        cache = LruCache(max_count)
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            nonlocal cache
            arguments = ""
            for arg in args:
                arguments = arguments + str(arg) + " "
            for key, val in kwargs.items():
                arguments = arguments + str(key) + "=" + str(value) + " "
            try:
                value = cache[arguments]
            except KeyError:
                cache[arguments] = func(*args, **kwargs)
                value = cache[arguments]
            return value
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [73]:
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(20)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [86]:
for i in range(50, 1050, 50):
    if (i > 900):
        print("i:{}, fact1(i):{}\n\n".format(i, fact1(i)))
    else:
        fact1(i)

i:950, fact1(i):139848254178293564380407669156048088991599844015581728918442436147108855483194542676962779356442218241705510616468732896880033783801509209190242670806258575457575876184062910114802220208861679191248905348286684536849268936268034865379620548355537385459245621807069469170552574039679870594086864672482428420709315161110957611072054844432931771021611064787799189622642249152185082859554040485802207594967733613588838222366726152022730774659027822584936482281868883625746305422778093923045110353871021585198720567753704848216022357219927928411745122413237300128337130132774882972029452019730417076771201454886158925000684539947649521389906345165615765004695918348675588148652191799146535313617413257208515955429430874234399760737511007455833662116186566964123389098452694461289857790768578913748989398314870922709696873762356689604024226320733567596358592275450870163757111721047156425277057075098613395786406785464246133305544689735625615022775970516114434207499808169081284568343783183

In [85]:
for i in range(50, 1050, 50):
    if (i > 900):
        print("i:{}, fact(i):{}\n\n".format(i, fact(i)))
    else:
        fact(i)

i:950, fact(i):1398482541782935643804076691560480889915998440155817289184424361471088554831945426769627793564422182417055106164687328968800337838015092091902426708062585754575758761840629101148022202088616791912489053482866845368492689362680348653796205483555373854592456218070694691705525740396798705940868646724824284207093151611109576110720548444329317710216110647877991896226422491521850828595540404858022075949677336135888382223667261520227307746590278225849364822818688836257463054227780939230451103538710215851987205677537048482160223572199279284117451224132373001283371301327748829720294520197304170767712014548861589250006845399476495213899063451656157650046959183486755881486521917991465353136174132572085159554294308742343997607375110074558336621161865669641233890984526944612898577907685789137489893983148709227096968737623566896040242263207335675963585922754508701637571117210471564252770570750986133957864067854642461333055446897356256150227759705161144342074998081690812845683437831834

RecursionError: maximum recursion depth exceeded in comparison

#### Сравните свою реализацию с lru_cache из functools

In [89]:
from functools import lru_cache

In [90]:
@lru_cache(20)
def fact2(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [92]:
for i in range(50, 1050, 50):
    if (i > 900):
        print("i:{}, fact2(i):{}\n\n".format(i, fact2(i)))
    else:
        fact2(i)

i:950, fact2(i):139848254178293564380407669156048088991599844015581728918442436147108855483194542676962779356442218241705510616468732896880033783801509209190242670806258575457575876184062910114802220208861679191248905348286684536849268936268034865379620548355537385459245621807069469170552574039679870594086864672482428420709315161110957611072054844432931771021611064787799189622642249152185082859554040485802207594967733613588838222366726152022730774659027822584936482281868883625746305422778093923045110353871021585198720567753704848216022357219927928411745122413237300128337130132774882972029452019730417076771201454886158925000684539947649521389906345165615765004695918348675588148652191799146535313617413257208515955429430874234399760737511007455833662116186566964123389098452694461289857790768578913748989398314870922709696873762356689604024226320733567596358592275450870163757111721047156425277057075098613395786406785464246133305544689735625615022775970516114434207499808169081284568343783183

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [ ]:
<your code here>

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [7]:
def checked(*types):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            arg_types = Counter(types)
            for arg in args:
                if (arg_types[type(arg)] <= 0):
                    raise TypeError
                arg_types[type(arg)] -= 1
            for _, value in kwargs.items():
                if (arg_types[type(value)] <= 0):
                    raise TypeError
                arg_types[type(value)] -= 1
            return func(*args, **kwargs)
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [8]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [9]:
strange_func('hello', 'l', ['4'])

TypeError: 

In [10]:
strange_func('hello', 7, ['4'])

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [60]:
import sys
import time

In [81]:
class Logger(object):
    dest = None
    counter = 0
    def __init__(self, file):     
        dest = file
        
    def write(self, func_name, func_args, func_kwargs, result):
        Logger.counter += 1
        cur_date = time.strftime("%d/%m/%y")
        cur_time = time.strftime("%H:%M:%S")
        
        print(Logger.counter, cur_date, cur_time, func_name, func_args, func_kwargs, result, 
              sep='  ', file=Logger.dest)

    def __call__(*args, **kwargs):
        

In [82]:
def logger(dest=sys.stdout):
    log = None
    def decorator(func):
        nonlocal log
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            nonlocal log
            if (log is None):
                log = Logger(dest)
            
            value = func(*args, **kwargs)
            log.write(func.__name__, args, kwargs, value)
            return value
           
        return wrapper
    return decorator

In [83]:
def hello(name):
    @logger(sys.stdout)
    def greet(greeting):
        return "{}, {}!".format(greeting, name)
    return greet

In [84]:
greet = hello("Natasha")
greet("Hello")
greet("Hola")
greet("Salute")

1  15/10/17  22:53:09  greet  ('Hello',)  {}  Hello, Natasha!
2  15/10/17  22:53:09  greet  ('Hola',)  {}  Hola, Natasha!
3  15/10/17  22:53:09  greet  ('Salute',)  {}  Salute, Natasha!


'Salute, Natasha!'